In [ ]:
import numpy as np
from collections import deque, namedtuple
import torch 
from torch import nn
import torch.distributions as dist
import random
import gymnasium as gym
import torch.optim as optim
from tqdm import trange
from scipy.stats import multivariate_normal



In [132]:
class Agent(object):
    ''' Base agent class

        Args:
            n_actions (int): actions dimensionality

        Attributes:
            n_actions (int): where we store the dimensionality of an action
    '''
    def __init__(self, n_actions: int):
        self.n_actions = n_actions

    def forward(self, state: np.ndarray):
        ''' Performs a forward computation '''
        pass

    def backward(self):
        ''' Performs a backward pass on the network '''
        pass


class RandomAgent(Agent):
    ''' Agent taking actions uniformly at random, child of the class Agent'''
    def __init__(self, n_actions: int):
        super(RandomAgent, self).__init__(n_actions)

    def forward(self, state: np.ndarray) -> np.ndarray:
        ''' Compute a random action in [-1, 1]

            Returns:
                action (np.ndarray): array of float values containing the
                    action. The dimensionality is equal to self.n_actions from
                    the parent class Agent
        '''
        return np.clip(-1 + 2 * np.random.rand(self.n_actions), -1, 1)


In [133]:
class CriticNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(8, 400),
            nn.ReLU(),
            nn.Linear(400, 200),
            nn.ReLU(),
            nn.Linear(200, 1),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

class ActorNetwork(nn.Module):
    def __init__(self, n_actions: int):
        super().__init__()
        self.flatten = nn.Flatten()
        self.relu = nn.ReLU()         
        self.tanh = nn.Tanh()
        self.sigmoid = nn.Sigmoid()
        self.Ls = nn.Linear(8,400)
        self.mu_head1 = nn.Linear(400, 200)
        self.mu_head2 = nn.Linear(200, n_actions)
        self.sigma_head1 = nn.Linear(400, 200)
        self.sigma_head2 = nn.Linear(200, n_actions)

    def forward(self, x):
        x = self.flatten(x)
        x = self.Ls(x)
        x = self.relu(x)
        # mu head
        mu = self.mu_head1(x)
        mu = self.relu(mu)
        mu = self.mu_head2(mu)
        mu = self.tanh(mu)
        # sigma head
        sigma = self.sigma_head1(x)
        sigma = self.relu(sigma)
        sigma = self.sigma_head2(sigma)
        sigma = self.sigmoid(sigma)

        return mu, sigma
 
class PPOAgent(Agent):

    def __init__(self, n_actions: int):
        super(PPOAgent, self).__init__(n_actions)
        self.critic_net = CriticNetwork()
        self.actor_net = ActorNetwork(n_actions)

    def forward(self, state: np.ndarray) -> np.ndarray:
        mu, sigma = self.actor_net(torch.tensor(np.expand_dims(state, axis=0)))
        a1 = mu[0][0].item() + sigma[0][0].item() * np.random.normal()
        a2 = mu[0][1].item() + sigma[0][1].item() * np.random.normal()
        return np.clip(np.array([a1, a2], dtype=np.float32), -1, 1)


In [134]:
# Define Experience tuple
# Experience represents a transition in the environment, including the current state, action taken,
# received reward, next state, and whether the episode is done.
Experience = namedtuple('Experience', ['state', 'action', 'reward', 'next_state', 'done'])

class ExperienceReplayBuffer:
    """Replay buffer for storing experiences.
    
       The experience replay buffer stores past experiences so that the agent can learn from them later.
       By sampling randomly from these experiences, the agent avoids overfitting to the most recent 
       transitions and helps stabilize training.
       - The buffer size is limited, and older experiences are discarded to make room for new ones.
       - Experiences are stored as tuples of (state, action, reward, next_state, done).
       - A batch of experiences is sampled randomly during each training step for updating the Q-values."""

    def __init__(self, maximum_length):
        self.buffer = deque(maxlen=maximum_length)  # Using deque ensures efficient removal of oldest elements

    def append(self, experience):
        """Add a new experience to the buffer"""
        self.buffer.append(experience)

    def __len__(self):
        """Return the current size of the buffer"""
        return len(self.buffer)

    def sample_batch(self, n):
        """Randomly sample a batch of experiences"""
        if n > len(self.buffer):
            raise IndexError('Sample size exceeds buffer size!')
        indices = np.random.choice(len(self.buffer), size=n, replace=False)  # Random sampling
        batch = [self.buffer[i] for i in indices]  # Create a batch from sampled indices
        return zip(*batch)  # Unzip batch into state, action, reward, next_state, and done


In [ ]:
N_EPISODES = 250  # Number of training episodes
BUFFER_SIZE = 10000  # Size of the replay buffer
ACTOR_LEARNING_RATE = 0.001  # Learning rate for the optimizer
CRITIC_LEARNING_RATE = 0.001 
GAMMA = 0.9
M=10

In [136]:
env = gym.make('LunarLanderContinuous-v3')
m = len(env.action_space.high)
agent = PPOAgent(m)
buffer = ExperienceReplayBuffer(maximum_length=BUFFER_SIZE)
actor_optimizer = optim.Adam(agent.actor_net.parameters(), lr=ACTOR_LEARNING_RATE)
critic_optimizer = optim.Adam(agent.critic_net.parameters(), lr=CRITIC_LEARNING_RATE)


In [137]:
def select_action(state, epsilon):
    """Epsilon-greedy action selection
    # We balance exploration and exploitation using epsilon-greedy.
    # Exploration: Choose a random action.
    # Exploitation: Choose the action with the highest Q-value (the optimal action)."""
    if random.random() < epsilon:
        return env.action_space.sample()  # Explore by selecting a random action
    else:
        state_tensor = torch.tensor([state], dtype=torch.float32)  # Convert state to tensor
        return agent.net(state_tensor).argmax().item()  # Exploit by selecting the action with max Q-value

# state = env.observation_space.sample()
# print("Type of sampled state:", type(state))
# print("Sampled state shape:", np.shape(state))

#s = torch.tensor(np.expand_dims(state, axis=0))

In [ ]:
from audioop import reverse

from torch import mul


EPISODES = trange(N_EPISODES, desc='Episode: ', leave=True)

for i in EPISODES:
    # Reset environment data
    done, truncated = False, False
    state = env.reset()[0]
    total_episode_reward = 0.0
    t = 0

    while not (done or truncated):
        action = agent.forward(state)
        next_state, reward, terminal, truncated, _ = env.step(action)
        done = terminal or truncated
        total_episode_reward += reward
        buffer.append(Experience(state, action, reward, next_state, done))
        state = next_state
        t += 1

    # Batch tensors for the collected trajectory
    states = torch.tensor(np.stack([exp.state for exp in buffer.buffer]), dtype=torch.float32)
    actions = torch.tensor(np.stack([exp.action for exp in buffer.buffer]), dtype=torch.float32)
    rewards = torch.tensor([exp.reward for exp in buffer.buffer], dtype=torch.float32)
    dones = torch.tensor([exp.done for exp in buffer.buffer], dtype=torch.float32)

    # Discounted returns (vectorized over the episode)
    returns = []
    running = 0.0
    for r, d in zip(reversed(rewards.tolist()), reversed(dones.tolist())):
        running = r + GAMMA * running * (1.0 - d)
        returns.append(running)
    returns = torch.tensor(list(reversed(returns)), dtype=torch.float32)

    with torch.no_grad():
        values = agent.critic_net(states).squeeze(-1)
        mu, sigma = agent.actor_net(states)
        dist = torch.distributions.Normal(mu, sigma)
        pi_old_logpdf = dist.log_prob(actions).sum(dim=1)

    advantages = returns - values.detach()

    # Placeholder for PPO update step using returns, advantages, and pi_old_logpdf
    # for _ in range(M):
    #     ... add actor/critic loss and optimizer steps ...

        



        
        



    #for i in reversed(range(t)):




        







Episode:   8%|▊         | 19/250 [00:06<01:13,  3.13it/s]


KeyboardInterrupt: 

In [ ]:
# env.action_space.sample()
# state = env.observation_space.sample()
# print("Type of sampled state:", type(state))
# print("Sampled state shape:", np.shape(state))

# s = torch.tensor(np.expand_dims(state, axis=0))

# mu[0][0].item()
# mu, sigma = agent.actor_net(s)
# agent.forward(state)

In [ ]:
a = Experience(0.1, 0.2, 0.3, 0.4, 0.5)
b = Experience(0.1, 0.2, 0.3, 0.4, 0.7)

for i, exp in zip(reversed(range(len([a, b]))), reversed([a, b])):
    print(exp)
    print(i)

Experience(state=0.1, action=0.2, reward=0.3, next_state=0.4, done=0.7)
1
Experience(state=0.1, action=0.2, reward=0.3, next_state=0.4, done=0.5)
0


In [ ]:
l = [0.1, 0.2, 0.3]

In [ ]:
l.reverse()


In [ ]:
l

[0.1, 0.2, 0.3]

In [ ]:
env.action_space.sample()
state = env.observation_space.sample()
print("Type of sampled state:", type(state))
print("Sampled state shape:", np.shape(state))

s = torch.tensor(np.expand_dims(state, axis=0))

#ActorNetworkmu[0][0].item()
mu, sigma = agent.actor_net(s)
agent.forward(state)

Type of sampled state: <class 'numpy.ndarray'>
Sampled state shape: (8,)


In [ ]:
logpdf.item()

-2.7282395362854004

array([-99.09631038,  99.60542318])

In [ ]:
multivariate_normal(np.array([100, -100]), np.diag([1,1]))

In [ ]:
multivariate_normal.logpdf([1, 1])

array([-1.41893853, -1.41893853])